In [9]:
import pandas as pd

class KlineAggregator:
    def __init__(self, window_minutes=5):
        """
        K 线数据整合器，实时接收数据并动态计算最近 `window_minutes` 分钟的 K 线。
        :param window_minutes: int, 目标 K 线周期（默认 5 分钟）
        """
        self.window_minutes = window_minutes
        self.data_buffer = []  # 缓存所有秒级数据
        self.latest_kline = None  # 记录当前最新 5 分钟 K 线

    def add_new_kline(self, new_kline):
        """
        添加新的 K 线数据，并计算最近 `window_minutes` 的 K 线
        :param new_kline: dict, 包含 'timestamp', 'open', 'high', 'low', 'close', 'vol'
        """
        self.data_buffer.append(new_kline)  # 存入缓存
        self.update_aggregated_kline(new_kline['timestamp'])  # 计算新的 5 分钟 K 线

    def update_aggregated_kline(self, current_time):
        """
        计算最近 `window_minutes` 的 K 线，动态更新
        :param current_time: 当前时间，决定 5 分钟窗口的范围
        """
        df = pd.DataFrame(self.data_buffer)
        df['timestamp'] = pd.to_datetime(df['timestamp'])
        df = df.sort_values(by='timestamp')

        # 计算当前 5 分钟窗口的起点（当前时间 - 5 分钟）
        start_time = pd.to_datetime(current_time) - pd.Timedelta(minutes=self.window_minutes)

        # 取最近 5 分钟的数据
        df_filtered = df[df['timestamp'] >= start_time]

        if df_filtered.empty:
            return  # 可能数据还不够，不计算

        # 计算 5 分钟 K 线
        open_price = df_filtered.iloc[0]['open']
        high_price = df_filtered['high'].max()
        low_price = df_filtered['low'].min()
        close_price = df_filtered.iloc[-1]['close']
        volume = df_filtered['vol'].sum()

        self.latest_kline = {
            'timestamp': current_time,  # 5 分钟K线的结束时间
            'open': open_price,
            'high': high_price,
            'low': low_price,
            'close': close_price,
            'vol': volume
        }

    def get_latest_kline(self):
        """获取最新的 5 分钟 K 线"""
        return self.latest_kline


In [11]:
# 生成 10 分钟的秒级数据
data = {
    "timestamp": pd.date_range("2025-01-01 18:00:00", periods=600, freq="s"),  # 1 秒间隔数据
    "open": [100 + i * 0.01 for i in range(600)],   # 递增 0.01
    "high": [102 + i * 0.01 for i in range(600)],   # 递增 0.01
    "low": [98 + i * 0.01 for i in range(600)],     # 递增 0.01
    "close": [101 + i * 0.01 for i in range(600)],  # 递增 0.01
    "vol": [1000 + i * 10 for i in range(600)]      # 递增 10
}
df_test = pd.DataFrame(data)

# 计算标准 5 分钟 K 线（Ground Truth）
df_test['timestamp'] = pd.to_datetime(df_test['timestamp'])
df_test.set_index('timestamp', inplace=True)

ground_truth_kline = df_test.resample('5min').agg({
    'open': 'first',
    'high': 'max',
    'low': 'min',
    'close': 'last',
    'vol': 'sum'
}).dropna().reset_index()

print("\n📌 标准 5 分钟 K 线（Ground Truth）:")
print(ground_truth_kline)



📌 标准 5 分钟 K 线（Ground Truth）:
            timestamp   open    high    low   close      vol
0 2025-01-01 18:00:00  100.0  104.99   98.0  103.99   748500
1 2025-01-01 18:05:00  103.0  107.99  101.0  106.99  1648500


In [12]:
# 创建 K 线整合器
aggregator = KlineAggregator(window_minutes=5)
generated_kline = []

# 逐步输入数据
for i in range(len(df_test)):
    new_kline = df_test.iloc[i].to_dict()
    aggregator.add_new_kline(new_kline)

    # 每隔 5 分钟保存一次
    if new_kline['timestamp'].second == 0 and new_kline['timestamp'].minute % 5 == 0:
        generated_kline.append(aggregator.get_latest_kline())

# 转换为 DataFrame 方便对比
generated_kline_df = pd.DataFrame(generated_kline)

print("\n📌 我们的 5 分钟 K 线:")
print(generated_kline_df)


KeyError: 'timestamp'

In [14]:
import pandas as pd

class KlineAggregator:
    def __init__(self, window_minutes=5):
        """K 线数据整合器，实时接收数据并动态计算最近 `window_minutes` 分钟的 K 线。"""
        self.window_minutes = window_minutes
        self.data_buffer = []  # 存储秒级数据
        self.latest_kline = None  # 记录最新 5 分钟 K 线

    def add_new_kline(self, new_kline):
        """添加新的秒级 K 线数据，并计算最近 `window_minutes` 的 K 线"""
        self.data_buffer.append(new_kline)
        self.update_aggregated_kline(new_kline['timestamp'])

    def update_aggregated_kline(self, current_time):
        """计算最近 `window_minutes` 的 K 线"""
        df = pd.DataFrame(self.data_buffer)
        df['timestamp'] = pd.to_datetime(df['timestamp'])
        df = df.sort_values(by='timestamp')

        # 计算当前 5 分钟窗口的起点（当前时间 - 5 分钟）
        start_time = pd.to_datetime(current_time) - pd.Timedelta(minutes=self.window_minutes)
        df_filtered = df[df['timestamp'] >= start_time]

        if df_filtered.empty:
            return

        self.latest_kline = {
            'timestamp': current_time,
            'open': df_filtered.iloc[0]['open'],
            'high': df_filtered['high'].max(),
            'low': df_filtered['low'].min(),
            'close': df_filtered.iloc[-1]['close'],
            'vol': df_filtered['vol'].sum()
        }

    def get_latest_kline(self):
        """获取最新的 5 分钟 K 线"""
        return self.latest_kline


# =============== 1. 生成 600 秒的秒级数据 ===============
initial_time = pd.Timestamp("2025-01-01 18:00:00")

data = {
    "timestamp": pd.date_range(initial_time, periods=600, freq="s"),  # 1 秒间隔数据
    "open": [100 + i * 0.01 for i in range(600)],   # 递增 0.01
    "high": [102 + i * 0.01 for i in range(600)],   # 递增 0.01
    "low": [98 + i * 0.01 for i in range(600)],     # 递增 0.01
    "close": [101 + i * 0.01 for i in range(600)],  # 递增 0.01
    "vol": [1000 + i * 10 for i in range(600)]      # 递增 10
}
df_test = pd.DataFrame(data)

# =============== 2. 计算 600 个 Ground Truth 5 分钟 K 线 ===============
df_test['timestamp'] = pd.to_datetime(df_test['timestamp'])
df_test.set_index('timestamp', inplace=True)

# **动态计算每一秒的 5 分钟窗口**
ground_truth_kline = []

for i in range(len(df_test)):
    current_time = df_test.index[i]  # 取当前秒的时间
    start_time = current_time - pd.Timedelta(minutes=5)  # 计算 5 分钟窗口的起点
    df_filtered = df_test[df_test.index >= start_time]  # 取 5 分钟窗口的数据

    ground_truth_kline.append({
        'timestamp': current_time,
        'open': df_filtered.iloc[0]['open'],
        'high': df_filtered['high'].max(),
        'low': df_filtered['low'].min(),
        'close': df_filtered.iloc[-1]['close'],
        'vol': df_filtered['vol'].sum()
    })

ground_truth_kline_df = pd.DataFrame(ground_truth_kline)

print("\n📌 标准 600 个 5 分钟 K 线（Ground Truth）:")
print(ground_truth_kline_df)

# =============== 3. 用 KlineAggregator 计算 600 个 5 分钟 K 线 ===============
aggregator = KlineAggregator(window_minutes=5)
generated_kline = []

for i in range(len(df_test)):
    new_kline = df_test.iloc[i].to_dict()
    aggregator.add_new_kline(new_kline)

    # 每秒存储当前计算出的 5 分钟 K 线
    generated_kline.append(aggregator.get_latest_kline())

generated_kline_df = pd.DataFrame(generated_kline)

print("\n📌 我们的 600 个 5 分钟 K 线:")
print(generated_kline_df)

# =============== 4. 进行结果对比 ===============
generated_kline_df = generated_kline_df[['timestamp', 'open', 'high', 'low', 'close', 'vol']]
ground_truth_kline_df = ground_truth_kline_df[['timestamp', 'open', 'high', 'low', 'close', 'vol']]

# 计算误差
comparison = ground_truth_kline_df.copy()
comparison['generated_open'] = generated_kline_df['open']
comparison['generated_high'] = generated_kline_df['high']
comparison['generated_low'] = generated_kline_df['low']
comparison['generated_close'] = generated_kline_df['close']
comparison['generated_vol'] = generated_kline_df['vol']

comparison['error_open'] = comparison['open'] - comparison['generated_open']
comparison['error_high'] = comparison['high'] - comparison['generated_high']
comparison['error_low'] = comparison['low'] - comparison['generated_low']
comparison['error_close'] = comparison['close'] - comparison['generated_close']
comparison['error_vol'] = comparison['vol'] - comparison['generated_vol']

print("\n📌 误差对比:")
print(comparison)

# =============== 5. 检查是否完全匹配 ===============
if comparison[['error_open', 'error_high', 'error_low', 'error_close', 'error_vol']].abs().sum().sum() == 0:
    print("\n✅ K 线计算完全正确，无误差！")
else:
    print("\n❌ K 线计算有误，请检查代码！")



📌 标准 600 个 5 分钟 K 线（Ground Truth）:
              timestamp    open    high     low   close      vol
0   2025-01-01 18:00:00  100.00  107.99   98.00  106.99  2397000
1   2025-01-01 18:00:01  100.00  107.99   98.00  106.99  2397000
2   2025-01-01 18:00:02  100.00  107.99   98.00  106.99  2397000
3   2025-01-01 18:00:03  100.00  107.99   98.00  106.99  2397000
4   2025-01-01 18:00:04  100.00  107.99   98.00  106.99  2397000
..                  ...     ...     ...     ...     ...      ...
595 2025-01-01 18:09:55  102.95  107.99  100.95  106.99  1668350
596 2025-01-01 18:09:56  102.96  107.99  100.96  106.99  1664400
597 2025-01-01 18:09:57  102.97  107.99  100.97  106.99  1660440
598 2025-01-01 18:09:58  102.98  107.99  100.98  106.99  1656470
599 2025-01-01 18:09:59  102.99  107.99  100.99  106.99  1652490

[600 rows x 6 columns]


KeyError: 'timestamp'

In [16]:
import pandas as pd
import numpy as np

class BollingerBandStrategy:
    def __init__(self, bb_window=20, bb_std_mult=3, fee_rate=0.001):
        """
        初始化布林带交易策略
        :param bb_window: 布林带计算的移动平均窗口
        :param bb_std_mult: 布林带标准差倍数
        :param fee_rate: 交易手续费率
        """
        self.bb_window = bb_window
        self.bb_std_mult = bb_std_mult
        self.fee_rate = fee_rate

    def calculate_bollinger_bands(self, df):
        """ 计算布林带指标 """
        df = df.copy()
        df['ma'] = df['close'].rolling(self.bb_window).mean()
        df['std'] = df['close'].rolling(self.bb_window).std(ddof=0)
        df['upper'] = df['ma'] + df['std'] * self.bb_std_mult
        df['lower'] = df['ma'] - df['std'] * self.bb_std_mult
        return df

    def generate_signal(self, df):
        """ 生成交易信号 """
        if len(df) < self.bb_window + 2:  # 确保至少有足够的数据
            return 0

        df = self.calculate_bollinger_bands(df)

        current = df.iloc[-1]
        prev = df.iloc[-2]

        # 做空信号：上轨突破
        if prev['high'] < prev['upper'] and current['high'] >= current['upper']:
            return -1

        # 做多信号：下轨突破
        if prev['low'] > prev['lower'] and current['low'] <= current['lower']:
            return 1

        return 0

    def check_exit_conditions(self, position_info, current_candle):
        """ 检查是否需要平仓 """
        entry_price = position_info['entry_price']
        direction = position_info['direction']
        position_size = position_info['position_size']

        if direction == 1:  # 多单
            unrealized_pnl = (current_candle['high'] - entry_price) * position_size
            close_fee = position_size * current_candle['high'] * self.fee_rate
        else:  # 空单
            unrealized_pnl = (entry_price - current_candle['low']) * position_size
            close_fee = position_size * current_candle['low'] * self.fee_rate

        net_pnl = unrealized_pnl - position_info['open_fee'] - close_fee

        # 止盈检查
        if net_pnl >= position_info['target_profit']:
            return True

        # 止损检查
        if unrealized_pnl <= -abs(position_info['stop_loss']):
            return True

        return False


class TradingBot:
    def __init__(self, strategy):
        """
        初始化交易机器人
        :param strategy: 交易策略对象（BollingerBandStrategy）
        """
        self.strategy = strategy
        self.position_info = None  # 当前持仓信息

    def process_new_kline(self, df):
        """
        处理新K线数据
        :param df: pandas DataFrame，包含最新K线数据
        """
        if self.position_info is None:
            # **检查是否需要开仓**
            signal = self.strategy.generate_signal(df)
            if signal != 0:
                self.open_position(signal, df.iloc[-1])
        else:
            # **检查是否需要平仓**
            should_exit = self.strategy.check_exit_conditions(self.position_info, df.iloc[-1])
            if should_exit:
                self.close_position()

    def open_position(self, signal, current_candle):
        """
        开仓逻辑
        :param signal: 1（做多）或 -1（做空）
        :param current_candle: 当前K线
        """
        self.position_info = {
            'direction': signal,
            'entry_price': current_candle['close'],
            'position_size': 10,  # 设定默认持仓大小
            'open_fee': 0.1,
            'stop_loss': 5,
            'target_profit': 10
        }
        print(f"开仓: {self.position_info}")

    def close_position(self):
        """ 平仓逻辑 """
        print(f"平仓: {self.position_info}")
        self.position_info = None  # 清除持仓


# === 运行示例 ===
if __name__ == "__main__":
    # 生成模拟K线数据
    np.random.seed(42)
    data = {
        'close': np.random.rand(30) * 100,  # 随机收盘价
        'high': np.random.rand(30) * 100 + 5,  # 随机最高价
        'low': np.random.rand(30) * 100 - 5   # 随机最低价
    }
    df = pd.DataFrame(data)

    # 初始化策略和交易机器人
    strategy = BollingerBandStrategy(bb_window=20, bb_std_mult=3, fee_rate=0.001)
    bot = TradingBot(strategy)

    # 模拟实时数据输入
    for i in range(len(df)):
        bot.process_new_kline(df.iloc[:i+1])  # 每次传入至当前K线的数据


In [ ]:
import pandas as pd
import numpy as np

class BollingerBandStrategy:
    def __init__(self, bb_window=20, bb_std_mult=3, fee_open=0.0005, fee_close=0.0002):
        self.bb_window = bb_window  # 布林带周期
        self.bb_std_mult = bb_std_mult  # 标准差倍数
        self.fee_open = fee_open  # 开仓手续费（万分之五）
        self.fee_close = fee_close  # 平仓手续费（万分之二）

    def calculate_bollinger_bands(self, df):
        """计算布林带指标"""
        df = df.copy()
        df['ma'] = df['close'].rolling(self.bb_window).mean()
        df['std'] = df['close'].rolling(self.bb_window).std(ddof=0)
        df['upper'] = df['ma'] + df['std'] * self.bb_std_mult
        df['lower'] = df['ma'] - df['std'] * self.bb_std_mult
        return df

    def generate_signal(self, df):
        """生成交易信号：1（做多），-1（做空），0（无操作）"""
        if len(df) < self.bb_window + 2:
            return 0

        df = self.calculate_bollinger_bands(df)
        current = df.iloc[-1]
        prev = df.iloc[-2]

        if prev['high'] < prev['upper'] and current['high'] >= current['upper']:
            return -1  # 做空
        if prev['low'] > prev['lower'] and current['low'] <= current['lower']:
            return 1  # 做多

        return 0  # 无交易信号

    def calculate_tp_sl(self, account_balance, invest_ratio, signal, entry_price, tp_ratio=0.005, sl_ratio=0.01):
        """
        计算止盈止损点位：
        - 止盈 = 账户资金的 0.5%（tp_ratio）
        - 止损 = 账户资金的 1.0%（sl_ratio）
        - 每单投资金额 = 账户资金 * invest_ratio
        - 带入开仓 / 平仓手续费
        """
        # 计算本次交易的投资金额
        invest_amount = account_balance * invest_ratio  # 例如 10% 账户资金

        # 计算目标盈利 / 亏损
        target_profit = account_balance * tp_ratio  # 盈利 0.5% 账户资金
        target_loss = account_balance * sl_ratio  # 亏损 1% 账户资金

        # 计算开仓手数（投资金额 / 开仓价）
        position_size = invest_amount / entry_price

        # 计算手续费
        open_fee = invest_amount * self.fee_open  # 开仓手续费
        close_fee_profit = (invest_amount + target_profit) * self.fee_close  # 平仓手续费（止盈）
        close_fee_loss = (invest_amount - target_loss) * self.fee_close  # 平仓手续费（止损）

        # 计算止盈止损价格
        if signal == 1:  # 多单
            take_profit_price = entry_price + ((target_profit - open_fee - close_fee_profit) / position_size)
            stop_loss_price = entry_price - ((target_loss + open_fee + close_fee_loss) / position_size)
        elif signal == -1:  # 空单
            take_profit_price = entry_price - ((target_profit - open_fee - close_fee_profit) / position_size)
            stop_loss_price = entry_price + ((target_loss + open_fee + close_fee_loss) / position_size)
        else:
            take_profit_price = None
            stop_loss_price = None

        return take_profit_price, stop_loss_price

    def process_trade(self, df, account_balance, invest_ratio=0.1, tp_ratio=0.005, sl_ratio=0.01):
        """
        处理交易：
        - 计算交易信号
        - 计算止盈 / 止损点位
        """
        signal = self.generate_signal(df)

        if signal == 0:
            return 0, None, None  # 无操作

        entry_price = df.iloc[-1]['close']  # 以当前收盘价作为开仓价
        take_profit_price, stop_loss_price = self.calculate_tp_sl(
            account_balance, invest_ratio, signal, entry_price, tp_ratio, sl_ratio
        )

        return signal, take_profit_price, stop_loss_price


# === 运行示例 ===
if __name__ == "__main__":
    np.random.seed(42)
    data = {
        'close': np.random.rand(30) * 100,  # 随机收盘价
        'high': np.random.rand(30) * 100 + 5,  # 随机最高价
        'low': np.random.rand(30) * 100 - 5  # 随机最低价
    }
    df = pd.DataFrame(data)

    strategy = BollingerBandStrategy(bb_window=20, bb_std_mult=3)

    # 假设账户余额 100,000
    account_balance = 100000

    # 计算交易信号 & 止盈止损点
    signal, tp, sl = strategy.process_trade(df, account_balance)

    print(f"交易信号: {signal}")
    print(f"止盈价格: {tp}")
    print(f"止损价格: {sl}")


In [ ]:
import pandas as pd
import numpy as np

class BollingerStrategy:
    def __init__(self, 
                 initial_balance=10000,
                 leverage=10,
                 position_ratio=0.1,
                 open_fee_rate=0.0005,
                 close_fee_rate=0.0002,
                 take_profit_ratio=0.005,
                 stop_loss_ratio=0.01,
                 bb_window=20,
                 bb_std_mult=3):
        
        # 资金参数
        self.balance = initial_balance
        self.leverage = leverage
        self.position_ratio = position_ratio
        
        # 费用参数
        self.open_fee_rate = open_fee_rate
        self.close_fee_rate = close_fee_rate
        
        # 风险参数
        self.take_profit_ratio = take_profit_ratio  # 总资金的0.5%
        self.stop_loss_ratio = stop_loss_ratio      # 总资金的1%
        
        # 布林带参数
        self.bb_window = bb_window
        self.bb_std_mult = bb_std_mult
        
    def calculate_bollinger_bands(self, df):
        """计算布林带指标"""
        df = df.copy()
        df['ma'] = df['close'].rolling(self.bb_window).mean()
        df['std'] = df['close'].rolling(self.bb_window).std(ddof=0)
        df['upper'] = df['ma'] + df['std'] * self.bb_std_mult
        df['lower'] = df['ma'] - df['std'] * self.bb_std_mult
        return df

    def generate_signal(self, df):
        """
        生成交易信号和止盈止损价格
        返回: (signal, take_profit_price, stop_loss_price)
        """
        if len(df) < self.bb_window:
            return 0, None, None
        
        df = self.calculate_bollinger_bands(df)
        
        current = df.iloc[-1]
        prev = df.iloc[-2]
        signal = 0

        # 做空信号
        if prev['high'] < prev['upper'] and current['high'] >= current['upper']:
            signal = -1
            
        # 做多信号
        elif prev['low'] > prev['lower'] and current['low'] <= current['lower']:
            signal = 1
            
        else:
            return 0, None, None
        
        # 以当前收盘价作为开仓价格
        entry_price = current['close']
        
        # 避免除 0 错误
        if entry_price <= 0:
            return 0, None, None
        
        # 计算仓位
        position_value = self.balance * self.position_ratio * self.leverage
        position_size = position_value / entry_price
        
        # 计算手续费
        open_fee = position_size * entry_price * self.open_fee_rate
        
        # 计算目标盈利和最大亏损（基于总资金）
        target_profit = self.balance * self.take_profit_ratio
        max_loss = self.balance * self.stop_loss_ratio
        
        # 计算止盈止损价格
        if signal == 1:  # 做多
            take_profit_price = entry_price + ((target_profit - open_fee) / (position_size * (1 - self.close_fee_rate)))
            stop_loss_price = entry_price - ((max_loss + open_fee) / (position_size * (1 + self.close_fee_rate)))
        else:  # 做空
            take_profit_price = entry_price - ((target_profit - open_fee) / (position_size * (1 + self.close_fee_rate)))
            stop_loss_price = entry_price + ((max_loss + open_fee) / (position_size * (1 - self.close_fee_rate)))
        
        return signal, round(take_profit_price, 4), round(stop_loss_price, 4)

    def update_balance(self, new_balance):
        """更新账户资金（在平仓后调用）"""
        self.balance = new_balance




交易信号: 0
止盈价格: None
止损价格: None
